In [2]:
import tensorflow as tf
from kerashypetune import KerasGridSearch
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [42]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 64  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen, ))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    x = layers.Dropout(0.2)(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.concatenate([avg_pool, max_pool])
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "sigmoid_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir sigmoid_models
    !mkdir $model_path
    
    
    model.compile(tf.keras.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
62/63 [============================>.] - ETA: 0s - loss: 0.6752 - accuracy: 0.5760 - auc: 0.6607
Epoch 00001: val_auc improved from -inf to 0.78144, saving model to sigmoid_models/0001/0001.ckpt
63/63 [==============================] - 3s 47ms/step - loss: 0.6749 - accuracy: 0.5769 - auc: 0.6614 - val_loss: 0.6543 - val_accuracy: 0.6430 - val_auc: 0.7814
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.5069 - accuracy: 0.8220 - auc: 0.9348
Epoch 00002: val_auc improved from 0.78144 to 0.92212, saving model to sigmoid_models/0001/0002.ckpt
63/63 [==============================] - 3s 41ms/step - loss: 0.5069 - accuracy: 0.8220 - auc: 0.9348 - val_loss: 0.4230 - val_accuracy: 0.8780 - val_auc: 0.9221
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.2097 - accuracy: 0.9419 - auc: 0.9868
Epoch 00003: val_auc improved from 0.92212 to 0.94323, saving model to sigmoid_models/0001/0003.ckpt
63/63 [==============================] - 3s 40ms/ste

In [43]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))

print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))


print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.27038323879241943 +/- 0.017976545369922416
test avg acc:  0.9081999897956848 +/- 0.007222179231799547
test avg auc:  0.950394606590271 +/- 0.004762078499129707


val avg loss:  0.26189036667346954 +/- 0.026335901030617306
val avg acc:  0.9101999998092651 +/- 0.011034492350755792
val avg auc:  0.9534595489501954 +/- 0.008111038977185805


train avg loss:  0.023613759502768516 +/- 0.013142081393495101
train avg acc:  0.9974999785423279 +/- 0.002319754875833748
train avg auc:  0.999566400051117 +/- 0.00046021444420742477
